En el siguiente paso, determinaremos la  efectividad de un pipeline más adecuado para manjear los atributos categóricos y numéricos, buscando una robustez mayor del modelo. Llevaremos los datos a valores nuéricos en su transformación y con el apoyo de una regresión analizaremos los resultados.

In [18]:
import pandas as pd
import numpy as np


df = pd.read_csv('ECH_2019.csv')


In [20]:
test = df.sample(n=10)

train = df.drop(labels=test.index)

print(test.shape)
print(train.shape)

(10, 30)
(107861, 30)


In [21]:
train.drop(columns=['id_hogar', 'nper'], inplace=True)

In [22]:
train['barrio'] = train['barrio'].astype('category')

train['barrio'] = train['barrio'].cat.codes


In [23]:
X = train.drop('sueldo', axis=1)
y = train['sueldo']


### Este nuevo pipeline nos ayuda a crear una transformación más acertada de lo que buscamos con el modelo, sin centrarnos en categorías de rangos de sueldos, yendo así directo a una estimación de sueldo.

In [24]:
from sklearn.compose import make_column_selector

numericas_selector = make_column_selector(dtype_exclude=object)
categoricas_selector = make_column_selector(dtype_include=object)

numericas = numericas_selector(X)

categoricas = categoricas_selector(X)


In [25]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_encoder = OneHotEncoder(handle_unknown="ignore")
numerical_scaler = StandardScaler()

In [26]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer([
    ('one-hot-encoder', categorical_encoder, categoricas),
    ('standard_scaler', numerical_scaler, numericas)])

In [27]:
from lightgbm import LGBMRegressor
from sklearn.pipeline import make_pipeline

model = make_pipeline(transformer, LGBMRegressor())

In [28]:
from sklearn.model_selection import train_test_split

# Definimos los feautures de test y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

In [29]:
model.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['sexo', 'ascendencia',
                                                   'parentesco',
                                                   'estado_laboral',
                                                   'busca_otro_trabajo',
                                                   'actividad',
                                                   'asistencia alguna vez a '
                                                   'enseñanza',
                                                   'edu preescolar',
                                                   'edu primaria', 'edu media',
                                                   'edu técnica',
                                                   'edu univ o simil',
                              

In [30]:
y_pred = model.predict(X_test)

In [31]:
from sklearn.metrics import mean_squared_error

import numpy as np

rmse = np.sqrt(mean_squared_error(y_test,y_pred))

In [32]:
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

0.6216517135907098
0.5696983662139783


### Los resultados de este modelo son mayores a los creados anteriormente. De todas formas siguen siendo bajo como para realizar una predicción efectiva para su uso pero aún así podemos tener varios insights con esta información. Podrímos realizar un RandomSearchCV o GridSearchCV pero como vimos en casos anteriores los valores para este dataset no varian demasiado. 

### A continuación investigaremos las estimaciones para una serie de features elegidas como prueba y cómo estas se comparan con el valor original del dataset.

In [35]:
prediccion = model.predict(test)
prediccion = pd.DataFrame(prediccion,index=test.index,columns=['predecido'])
prediccion.head()

,predecido
88946,8434.126084
38463,24347.464541
40048,26787.044509
11666,6984.517212
71324,8067.914863


In [36]:
prediccion['original'] = [df['sueldo'].loc[i] for i in prediccion.index]

prediccion

,predecido,original
88946,8434.126084,0.0
38463,24347.464541,30000.0
40048,26787.044509,38000.0
11666,6984.517212,0.0
71324,8067.914863,22000.0
4776,30700.751563,35000.0
20574,37021.522304,43000.0
93438,61.492854,0.0
60333,293.598128,0.0
85724,8421.067101,12000.0


Los estimados difieren en algunos casos notoriamente con los datos originales pero aún así nos podría permitir un análisis superficial de la situación. Con este modelo creado podríamos comparar cómo afectaría un cambio de sexo en los casos monomarentales y determinar qué tanta diferencia existe entre cada uno, finalizando así la investigación.